In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import csv
import scipy.stats as stats
from datetime import datetime
from matplotlib import gridspec
from scipy.signal import argrelmax
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pywt

meal1 = pd.read_csv(r"./Processed_Data/mealData1.csv", error_bad_lines=False)
meal2 = pd.read_csv(r"./Processed_Data/mealData2.csv", error_bad_lines=False)
meal3 = pd.read_csv(r"./Processed_Data/mealData3.csv", error_bad_lines=False)
meal4 = pd.read_csv(r"./Processed_Data/mealData4.csv", error_bad_lines=False)
meal5 = pd.read_csv(r"./Processed_Data/mealData5.csv", error_bad_lines=False)

meal2.columns = meal1.columns
meal3.columns = meal1.columns
meal4.columns = meal1.columns
meal5.columns = meal1.columns

#meal dataframe - all five meal data files appended together
meal = pd.concat([meal1, meal2, meal3, meal4, meal5], ignore_index=True, sort = False)

no_meal1 = pd.read_csv(r"./Processed_Data/Nomeal1.csv", error_bad_lines=False)
no_meal2 = pd.read_csv(r"./Processed_Data/Nomeal2.csv", error_bad_lines=False)
no_meal3 = pd.read_csv(r"./Processed_Data/Nomeal3.csv", error_bad_lines=False)
no_meal4 = pd.read_csv(r"./Processed_Data/Nomeal4.csv", error_bad_lines=False)
no_meal5 = pd.read_csv(r"./Processed_Data/Nomeal5.csv", error_bad_lines=False)

no_meal2.columns = no_meal1.columns
no_meal3.columns = no_meal1.columns
no_meal4.columns = no_meal1.columns
no_meal5.columns = no_meal1.columns

#no meal dataframe - all five no meal data files appended together
no_meal = pd.concat([no_meal1, no_meal2, no_meal3, no_meal4, no_meal5], ignore_index=True, sort = False)


In [22]:
#Extract Monitoring CGM Trend
def extract_cgm_trend(df, result_df):

    lunch = [[]]
    means = []

    for i in range(1, len(df)):
        lunch.append(df.iloc[i])

    for i in range(0, len(lunch)):
        means.append(df.iloc[i].mean())

    countmaster = []

    for i in range(0, len(lunch)):
        count = 0

        for j in df.iloc[i]:
            if j < means[i]:    
                count += 1

        countmaster.append(count)

    percentage=[]
    for i in countmaster:
        percentage.append((i / len(df.iloc[0])) * 100)

    result_df['feature_2'] = np.asarray(percentage)
    
    #return result_df


In [23]:
#Extract Accelaration
def extract_acceleration(df, result_df):
    d=[]
    q=[]
    f=[]

    acc1 = []
    acc2 = []
    acc3 = []
    acc4 = []

    for j in range(0, df.shape[0]):
        b = df.iloc[j]
        d = []
        for i in range(len(b)):
            if(np.isnan(b[i])):
                continue
            else:
                d.append(b[i])

        if(len(d) >= 1):

            solar_elevation_angle_1stdev = np.gradient(d)

            solar_elevation_angle_2nddev = np.gradient(solar_elevation_angle_1stdev)

            q = solar_elevation_angle_2nddev
            arr = q[5:10]
            acc1.append(np.mean(arr))
            arr = q[10:15]
            acc2.append(np.mean(arr))
            arr = q[15:20]
            acc3.append(np.mean(arr))
            arr = q[20:25]
            acc4.append(np.mean(arr))

        else:
            acc1.append(0)
            acc2.append(0)
            acc3.append(0)
            acc4.append(0)

    result_df['acc1'] = acc1
    result_df['acc2'] = acc2
    result_df['acc3'] = acc3
    result_df['acc4'] = acc4
    
    #return result_df


In [24]:
#Extract Entropy
def SampEn(U, m, r):
    def _maxdist(x_i, x_j):
        result = max([abs(ua - va) for ua, va in zip(x_i, x_j)])
        return result

    def _phi(m):
        x = [[U[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = 0
        for i in range(len(x)):
            for j in range(len(x)):
                if i == j:
                    continue
                C += (_maxdist(x[i], x[j]) <= r)
        return C
    
    N = len(U)
    if(_phi(m+1) / _phi(m) == 0) :
        return 1
    return -np.log(float(_phi(m+1)) / float(_phi(m)))

def extract_entropy(df, result_df):
    no_of_rows = df.shape[0]
    no_of_cols = df.shape[1]
    
    result = df.T
    cols = []
    for i in range(0, no_of_rows) :
        cols.append(str(i))
    result.columns = cols 
    m = 2
    
    entropy = []
    for i in cols :
        r = 0.2 * np.std(result[i])
        entropy.append(SampEn(result[i], 2, r))

    result_df['entropy'] = entropy
    #return result_df

In [25]:
#Extract PolyFit
def extract_polyfit(df, result_df):
   
    colid = []
    no_of_rows = df.shape[0]
    no_of_cols = df.shape[1]

    result = df.T
    no_of_coefficients = 6
    cols = []
    for i in range(0, no_of_rows) :
        cols.append(str(i))
    result.columns = cols
    
    for i in range(0, no_of_cols) :
        colid.append(i + 1)
    x = np.array(colid)
    fftmatrix = []
    
    for i in range(0, no_of_rows) :
        y = np.array(result[str(i)])
        polyres = np.polyfit(x, y, no_of_coefficients-1)  
        fftmatrix.append(polyres.tolist())
    fftmatrix = np.array(fftmatrix)   
  
    for i in range(no_of_coefficients) :
        result_df['Coefficient'+str(i)] = fftmatrix[:, i]
    #return result_df


In [26]:
def extract_dwt(df, result_df):
    cA, cB = pywt.dwt(df, 'haar')
    cA_threshold = pywt.threshold(cA, np.std(cA)/2, mode='soft')
    cB_threshold = pywt.threshold(cB, np.std(cB)/2, mode='soft')
    
    reconstructed_signal = pywt.idwt(cA_threshold, cB_threshold, 'haar')
    
    feature_dwt_top8 = cA[:,:-8] #sorted in Ascending
    #print(feature_dwt_top8.shape)
    for i in range(8) :
        result_df['DWT'+str(i)] = feature_dwt_top8[:, i]

In [27]:
def zero_crossing(df, result_df):
    max_vel = []
    min_vel = []
    zero_cross = []
    max_ind = []
    min_ind = []
    data_list = df.values.tolist()
    
    x = len(data_list)
    y = len(data_list[0])
    diff = 5
    
    for i in range(0, x):
        max_v = -1000
        min_v = 1000000
        c = 0
        max_i = 0
        min_i = 0
        row = data_list[0]

In [28]:
#1. Feature extraction
meal_features = pd.DataFrame()
no_meal_features = pd.DataFrame()

def extract_features(data, result_df) : 
    extract_cgm_trend(data, result_df)
    extract_acceleration(data, result_df)
    extract_entropy(data, result_df)
    extract_polyfit(data, result_df)
    extract_dwt(data, result_df)

extract_features(meal, meal_features)
#print(meal_features)
extract_features(no_meal, no_meal_features)
#print(no_meal_features)

In [29]:
#2. PCA
pca = PCA(n_components=10)
principalComponents = pca.fit(meal_features)
PCA_mealdata = pca.transform(meal_features)
#print(PCA_mealdata)
PCA_nomealdata = pca.transform(no_meal_features)
#print(PCA_nomealdata)
Training_data = np.concatenate((PCA_mealdata, PCA_nomealdata), axis=0)

Training_labels = []
no_of_mealrows = meal.shape[0]
no_of_nomealrows = no_meal.shape[0]
for i in range(no_of_mealrows) :
    Training_labels.append(1)
for i in range(no_of_nomealrows) :
    Training_labels.append(0)


In [30]:
import pandas as pd
import numpy as np
import pickle
import csv

no_of_files = input("Input number of files to read: ")
file_paths = []
for i in range (0, int(no_of_files)):
    file_paths.append(input("Enter file path " + str(i + 1) + ": "))
    
test_df = pd.read_csv(file_paths[0], error_bad_lines=False)
    
for i in range(1, len(file_paths)):
    sample = pd.read_csv(file_paths[i], error_bad_lines=False)
    sample.columns = test_df.columns
    test_df = pd.concat([test_df, sample], ignore_index=True, sort = False)
    
#1. Test data - Feature extraction
test_features = pd.DataFrame()

def extract_features(test_df, test_features) : 
    extract_cgm_trend(test_df, test_features)
    extract_acceleration(test_df, test_features)
    extract_entropy(test_df, test_features)
    extract_polyfit(test_df, test_features)
    extract_dwt(test_df, test_features)

extract_features(test_df, test_features)
pca_test_data = pca.transform(test_features)    

filename = r"./Decision_Tree_Model.sav"
clf = pickle.load(open(filename, 'rb'))
y_pred = clf.predict(pca_test_data)
new_y_pred = []
for i in y_pred:
    if(i > 0):
        new_y_pred.append(1)
    else:
        new_y_pred.append(0)
        
with open('./Result_File.csv', 'w', newline='') as file:
    wr = csv.writer(file, quoting=csv.QUOTE_ALL)
    wr.writerow(new_y_pred)
    
print("----------------------Processing complete------------------------")

Input number of files to read: 1
Enter file path 1: /home/bhargavi/DM/DM_HW2/Processed_Data/Nomeal3.csv
----------------------Processing complete------------------------


/usr/local/lib/python3.5/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
